In [1]:
import netgen.gui
from ngsolve import *
import netgen.geom2d as geom2d

In [2]:
# next we define a function that solves the plate equation

def solve_plate(Compound_Prod_Space, t):
    
    # w and beta are trial functions
    (w, beta_x, beta_y) = Compound_Prod_Space.TrialFunction()
    # v and delta are test functions
    (v, delta_x, delta_y) = Compound_Prod_Space.TestFunction()
        
    # storing beta and delta as vectors
    beta = CoefficientFunction((beta_x,beta_y))
    delta = CoefficientFunction((delta_x, delta_y))

    # it is somehow not possible to take the grad of a vector, so define beta_grad component-wise
    beta_grad = CoefficientFunction((grad(beta_x)[0],grad(beta_y)[0]))
    delta_grad = CoefficientFunction((grad(delta_x)[0], grad(delta_y)[0]))

    A = BilinearForm(Compound_Prod_Space)
    A+= beta_grad * delta_grad * dx + 1/t**2 * (grad(w)-beta) * (grad(v)-delta) * dx
    A.Assemble()

    f = LinearForm(Compound_Prod_Space)
    f += 1 * v * dx
    f.Assemble()
    
    sol = GridFunction(Compound_Prod_Space)
    sol.vec.data = \
        A.mat.Inverse(freedofs=Compound_Prod_Space.FreeDofs(),inverse = "umfpack") * f.vec
    return sol

In [3]:
mesh = Mesh(geom2d.unit_square.GenerateMesh(maxh=0.1))
k = 1
t = 0.01

#different boundary conditions

# first boundary conditions
Sigma_1 = H1(mesh, order = k, dirichlet = "left|right|top|bottom") # BCs for w
V_1 = H1(mesh, order= k, dirichlet = "left|right|top|bottom") # BCs for beta, I'm not sure if this is right
FES_1 = FESpace([Sigma_1, V_1, V_1])

sol_1 = solve_plate(FES_1,t)
w_1 = sol_1.components[0]
beta_1 =  CoefficientFunction((sol_1.components[1], sol_1.components[2]))

Draw(w_1, mesh, "w_bc1")
Draw(beta_1, mesh, "beta_bc1")


#second boundary condtions
Sigma_2 = H1(mesh, order = k, dirichlet = "left|top")
V_2 = H1(mesh, order= k, dirichlet = "left|top")
FES_2 = FESpace([Sigma_2, V_2, V_2])

sol_2 = solve_plate(FES_2,t)
w_2 = sol_2.components[0]
beta_2 =  CoefficientFunction((sol_2.components[1], sol_2.components[2]))

Draw(w_2, mesh, "w_bc2")
Draw(beta_2, mesh, "beta_bc2")


# third boundary conditions
Sigma_3 = H1(mesh, order = k, dirichlet = "left|right|top|bottom")
V_3 = H1(mesh, order= k)
FES_3 = FESpace([Sigma_3, V_3, V_3])

sol_3 = solve_plate(FES_3,t)
w_3 = sol_3.components[0]
beta_3 =  CoefficientFunction((sol_3.components[1], sol_3.components[2]))

Draw(w_3, mesh, "w_bc3")
Draw(beta_3, mesh, "beta_bc3")